# Intro

**This notebook is highly based on amazing work done by DLASTSTARK, JAVIER VALLEJOS and PRIYANSHU CHAUDHARY, you may find link below:**

* https://www.kaggle.com/dlaststark/tps-1121-dnn-v4
* 
https://www.kaggle.com/javiervallejos/simple-nn-with-good-results-tps-nov-21
* https://www.kaggle.com/chaudharypriyanshu/understanding-neural-net


    
**Please give them fully deserved interest!** 👏

![](https://media.giphy.com/media/I4wGMXoi2kMDe/giphy-downsized-large.gif)

# Import libraries

In [ ]:
import gc
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import StratifiedKFold

import tensorflow as tf
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Input, BatchNormalization
from tensorflow.keras.layers import Dense, Dropout, Multiply

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

np.random.seed(42)
tf.random.set_seed(42)

# Load source datasets

In [ ]:
train_df = pd.read_csv("../input/tabular-playground-series-nov-2021/train.csv")
train_df.set_index('id', inplace=True)
print(f"train_df: {train_df.shape}")
train_df.head()

In [ ]:
test_df = pd.read_csv("../input/tabular-playground-series-nov-2021/test.csv")
test_df.set_index('id', inplace=True)
ind = test_df.index.tolist()
print(f"test_df: {test_df.shape}")
test_df.head()

# Feature Engineering
I was basing on DLASTSTARK feature engineering proposal but also added some statistics as : *'mean', 'std', 'min', 'max', 'median'*. During many tests found it worth to consider.

In [ ]:
mode = 'add'

if mode == 'add':
    
    features = test_df.columns.tolist() 

    for col in tqdm(features):
        train_df[col+'_bin'] = train_df[col].apply(lambda x: 1 if np.cbrt(x)>0 else 0) #1
        test_df[col+'_bin'] = test_df[col].apply(lambda x: 1 if np.cbrt(x)>0 else 0) #1
    
    print('Calc train features...')

    train_df["mean"] = train_df[features].mean(axis=1)
    train_df["std"] = train_df[features].std(axis=1)
    train_df["min"] = train_df[features].min(axis=1)
    train_df["max"] = train_df[features].max(axis=1)
    train_df["median"] = train_df[features].median(axis=1)


    
    print('Calc test features...')
    test_df["mean"] = test_df[features].mean(axis=1)
    test_df["std"] = test_df[features].std(axis=1)
    test_df["min"] = test_df[features].min(axis=1)
    test_df["max"] = test_df[features].max(axis=1)
    test_df["median"] = test_df[features].median(axis=1)


    features = test_df.columns.tolist()

    features.extend(['mean', 'std', 'min', 'max', 'median'])

    print(f"train_df: {train_df.shape} \ntest_df: {test_df.shape}")

else:
    pass

# Scaling / Dim Reduction / Whitening
Below you may switch between dim reduction / scaling / whitening methods prepared above. By default I did not use them.
If you are novice you may find some interesting information below:

1. PCA: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html
2. Quantile Transformation: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.QuantileTransformer.html
3. MinMaxScaler: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
4. Whitening: https://en.wikipedia.org/wiki/Whitening_transformation

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import QuantileTransformer

def PCA_transform(train, test):
    pca = PCA(n_components=16, whiten = False)
    train = train.drop(columns=['target']).values
    test = test.values
    train = pca.fit_transform(train)
    test = pca.transform(test)
    return train, test

def Q_transform(train, test):
    qtf = QuantileTransformer(random_state=21, n_quantiles=2000, output_distribution='normal')
    train = train.drop(columns=['target']).values
    test = test.values
    train = qtf.fit_transform(train)
    test = qtf.transform(test)
    return train, test

def svd_whiten(train, test):
    train = train.drop(columns=['target']).values
    test = test.values
    
    U, s, Vt = np.linalg.svd(train, full_matrices=False)
    train_arr = np.dot(U, Vt)
    
    U, s, Vt = np.linalg.svd(test, full_matrices=False)
    test_arr = np.dot(U, Vt)
    # U and Vt are the singular matrices, and s contains the singular values.
    # Since the rows of both U and Vt are orthonormal vectors, then U * Vt
    # will be white
    return train_arr, test_arr

def scale_data(train, test):
    feats_to_scale = ['f{}' for e in range(100)]
    feats_to_add = ['f{}_bin' for e in range(100)] + ['mean', 'std', 'min', 'max', 'median']
    
    
    train = train.drop(columns=['target']).values
    test = test.values
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_arr = scaler.fit_transform(train)
    test_arr = scaler.transform(test)

    return train_arr, test_arr

In [ ]:
mode = None

if mode == 'PCA':
    targets = train_df.target.values
    train_arr, test_arr = PCA_transform(train_df, test_df)
elif mode == 'Quantile':
    targets = train_df.target.values
    train_arr, test_arr = Q_transform(train_df, test_df)
elif mode == 'SVD':
    targets = train_df.target.values
    train_arr, test_arr = svd_whiten(train_df, test_df)
elif mode == 'SCALE':
    targets = train_df.target.values
    train_arr, test_arr = scale_data(train_df, test_df)
else:
    targets = train_df.target.values
    train_arr = train_df.drop(columns=['target']).values
    test_arr = test_df.values

In [ ]:
print(train_arr.shape, test_arr.shape)

In [ ]:
print(train_arr.max(), train_arr.min(), test_arr.max(), test_arr.min())

In [ ]:
train_df = pd.DataFrame(train_arr)
test_df = pd.DataFrame(test_arr)
features = train_df.shape[1]
print(f"Num features: {train_df.shape[1]}")

# Helper Function

In [ ]:
def plot_confusion_matrix(cm, classes):

    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix', fontweight='bold', pad=15)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=0)
    plt.yticks(tick_marks, classes)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], 'd'),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label', fontweight='bold')
    plt.xlabel('Predicted label', fontweight='bold')
    plt.tight_layout()

# Keras Model

Model structure was enhanced by an additional branch of CONV2D which was to extract additional features concatenated then with Enc/Dec outputs.

In [ ]:
from tensorflow.keras.layers import Reshape, Conv2D, GlobalMaxPooling2D, UpSampling2D, MaxPooling2D, Flatten, AveragePooling2D

def dnn_model():
    x_input = Input(shape=(features,))
    
    x1 = Dense(units=386, activation='swish')(x_input)
    x1 = BatchNormalization()(x1)
    x2 = Dropout(rate=0.45)(x1)
    
    x2 = Dense(units=192, activation='swish')(x2)
    x2 = BatchNormalization()(x2)
    x3 = Dropout(rate=0.35)(x2)
    
    x3 = Dense(units=98, activation='swish')(x3)
    x3 = BatchNormalization()(x3)
    x3 = Dropout(rate=0.25)(x3)
    
    x4 = Dense(units=192, activation='swish')(x3)
    x4 = BatchNormalization()(x4)
    x4 = Multiply()([x2, x4])
    x4 = Dropout(rate=0.35)(x4)
    
    x5 = Dense(units=386, activation='swish')(x4)
    x5 = BatchNormalization()(x5)
    x5 = Multiply()([x1, x5])
    x5 = Dropout(rate=0.45)(x5)
    
    x_e1 = Concatenate()([x3, x5])
    x = Reshape((22, 22, 1))(x_e1)
    
    x = Conv2D(16, kernel_size=(3, 3), padding='same', activation='swish')(x)#16
    x = AveragePooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, kernel_size=(3, 3), padding='same', activation='swish')(x)#8
    x = AveragePooling2D((2, 2), padding='same')(x)
    x = Conv2D(8, kernel_size=(3, 3), padding='same', activation='swish')(x)#8
    
    enc = AveragePooling2D((2, 2), padding='same')(x)
    
    flat = Flatten()(enc)
    
    x = Concatenate()([x_e1, flat])
    
    x = Dense(units=128, activation='swish')(x)
    x = BatchNormalization()(x)
    x = Dropout(rate=0.25)(x)
    
    x_output = Dense(units=1, activation='sigmoid')(x)

    model = Model(inputs=x_input, outputs=x_output, 
                  name='DNN_Model')
    return model

In [ ]:
model = dnn_model()

In [ ]:
from tensorflow.keras.utils import plot_model
plot_model(
    model, 
    to_file='DNN_CNN_Model.png', 
    show_shapes=True,
    show_layer_names=True
)

# UMAP VIS
Some sample visualisation by umap reduction in 3D space, found it useful to compare scaling methods.
If you are not familiar with umap check it out here: https://umap-learn.readthedocs.io/en/latest/basic_usage.html

In [ ]:
import plotly
import plotly.graph_objs as go
import numpy as np
import random
import umap

SAVE_FLAG = False
file_out = './default.html'

to_vis = pd.DataFrame(train_arr)
to_vis['label'] = targets
to_vis = to_vis.sample(n=5000)
labels = to_vis.label.tolist()

print(to_vis.label.value_counts())

X_emb = to_vis.drop(columns=['label'])

metric_umap = 'correlation'
adapted_embedded = umap.UMAP(n_neighbors=25,
                      n_components=3,
                      min_dist=0.5,
                      metric=metric_umap).fit_transform(X_emb)

adapted_embedded = pd.DataFrame(adapted_embedded, columns=['V1', 'V2', 'V3'])
adapted_embedded['to_view'] = ''
adapted_embedded['labels'] = labels
labels_list = adapted_embedded.labels.value_counts().index.tolist()



def plot_tsne(df:pd.DataFrame, label_list, to_save = False, add_custom_legend = False, file_out = 'default.html'):
    list_of_plots = []
    for i, lab in enumerate(label_list):
        adapted_embedded_f = df[df.labels == lab]
        color = random.randint(0, 0xFFFFFF)
        
        x1 = adapted_embedded_f['V1'].values
        y1 = adapted_embedded_f['V2'].values
        z1 = adapted_embedded_f['V3'].values
        

        plot_f = go.Scatter3d(
                x=x1,
                y=y1,
                z=z1,
                hovertext = adapted_embedded_f.to_view.tolist(),
                mode='markers',
                marker=dict(
                    size=4,
                    color=color,            
                    symbol='circle',
                    line=dict(
                            color=color,
                            width=1
                        )
                ),
                name=lab,
            )
        list_of_plots.append(plot_f)
    
    fig = go.Figure(data=list_of_plots)
    
    if add_custom_legend:
        fig.update_layout(
        legend=dict(
                x=0,
                y=1,
                traceorder="reversed",
                title_font_family="Calibri",
                font=dict(
                    family="Courier",
                    size=12,
                    color="black"
                ),
                bgcolor="White",
                bordercolor="Black",
                borderwidth=1
                )
            )
        
    plotly.offline.iplot(fig)
    if to_save:
        plotly.offline.plot(fig, filename=file_out)
        
plot_tsne(adapted_embedded, labels_list, to_save=SAVE_FLAG, file_out = file_out)

# Training and testing

Class weighting implemented mechanism is controled by CLASS_WEIGHT_FLAG, by default it's off (data is almost balanced).

In [ ]:
FOLD = 5
VERBOSE = 0
SEEDS = [2021, 2025]
BATCH_SIZE = 2048
CLASS_WEIGHT_FLAG = False

counter = 0
oof_score = 0
y_pred_final_dnn = np.zeros((test_df.shape[0], 1))
y_pred_meta_dnn = np.zeros((train_df.shape[0], 1))


for sidx, seed in enumerate(SEEDS):
    seed_score = 0
    
    kfold = StratifiedKFold(n_splits=FOLD, shuffle=True, random_state=seed)

    for idx, (train, val) in enumerate(kfold.split(train_arr, targets)):
        counter += 1
        
        #define train and validation data by indices
        train_x, train_y = train_arr[train], targets[train]
        val_x, val_y = train_arr[val], targets[val]
        
        #count zero and one label ratios
        zero = np.unique(train_y, return_counts=True)[1][0]
        one = np.unique(train_y, return_counts=True)[1][1]
        total = len(train_y)

        weight_for_0 = (1 / zero)*(total)/2.0 
        weight_for_1 = (1 / one)*(total)/2.0

        class_weight = {0: weight_for_0, 1: weight_for_1}

        print('Weight for class 0: {:.2f}'.format(weight_for_0))
        print('Weight for class 1: {:.2f}'.format(weight_for_1))
        
        #init and compile model
        model = dnn_model()
        model.compile(optimizer=Adam(learning_rate=1e-2), 
                      loss="binary_crossentropy", 
                      metrics=['AUC'])

        lr = ReduceLROnPlateau(monitor="val_loss", factor=0.2, 
                               patience=7, verbose=VERBOSE)
        
        chk_point = ModelCheckpoint(f'./Keras_DNN_Model_{counter}C.h5', 
                                    monitor='val_loss', verbose=VERBOSE, 
                                    save_best_only=True, mode='min')

        es = EarlyStopping(monitor="val_loss", patience=30, 
                           verbose=VERBOSE, mode="min", 
                           restore_best_weights=True)
        
        if CLASS_WEIGHT_FLAG == False:
            class_weight = None
        else:
            pass
        
        #train model
        model.fit(train_x, train_y, 
                  validation_data=(val_x, val_y), 
                  epochs=900,
                  verbose=VERBOSE,
                  class_weight=class_weight,
                  batch_size=BATCH_SIZE, 
                  callbacks=[lr, chk_point, es])
        
        #load model and perform tests
        model = load_model(f'./Keras_DNN_Model_{counter}C.h5')
        
        y_pred = model.predict(val_x, batch_size=BATCH_SIZE)
        y_pred_meta_dnn[val] += y_pred
        y_pred_final_dnn += model.predict(test_arr, batch_size=BATCH_SIZE)
        
        score = roc_auc_score(val_y, y_pred)
        oof_score += score
        seed_score += score
        print("\nSeed-{} | Fold-{} | OOF Score: {}\n".format(seed, idx, score))
        
        del model, y_pred
        del train_x, train_y
        del val_x, val_y
        gc.collect()
    
    print("\nSeed: {} | Aggregate OOF Score: {}\n\n".format(seed, (seed_score / FOLD)))


y_pred_meta_dnn = y_pred_meta_dnn / float(len(SEEDS))
y_pred_final_dnn = y_pred_final_dnn / float(counter)
oof_score /= float(counter)

In [ ]:
print("Aggregate OOF Score: {}".format(oof_score))

In [ ]:
y_pred_meta = np.mean(y_pred_meta_dnn, axis=1)
y_pred = (y_pred_meta>0.5).astype(int)
print(classification_report(targets, y_pred))

In [ ]:
cnf_matrix = confusion_matrix(targets, y_pred, labels=[0, 1])
np.set_printoptions(precision=2)
plt.figure(figsize=(12, 5))
plot_confusion_matrix(cnf_matrix, classes=[0, 1])

# Some weighted average ensemble


In [ ]:
def avg_results(l1, l2):
    en_l = []
    for i, e in enumerate(l1):
        r = (0.3 * e) + (0.7 * l2[i])
        en_l.append(r)
        
    return en_l

l1 = y_pred_final_dnn.ravel()
l2 = pd.read_csv('../input/simple-nn-with-good-results-tps-nov-21/submission.csv').target.tolist()

ens = avg_results(l1, l2)

df_vis = pd.DataFrame()
df_vis['model1'] = l1
df_vis['model2'] = l2
df_vis['model_ens'] = ens

In [ ]:
#DNN/CNN model prob dist
df_vis.model1.hist(bins=1000)

In [ ]:
#Simple NN model prob dist
df_vis.model2.hist(bins=1000)

In [ ]:
#Ens model prob dist
df_vis.model_ens.hist(bins=1000)

# Create Submission

In [ ]:
submit_df = pd.read_csv("../input/tabular-playground-series-nov-2021/sample_submission.csv")
submit_df['target'] = ens
submit_df.to_csv("submission.csv", index=False)
submit_df.head()